# <center>Лабораторная работа №6
## <center>Линейная регрессия, Lasso и RF-регрессия в задаче по определению качества вина

<img src='../../img/wine_quality.jpg' width=30%>

**Заполните пропущенный код.**

In [19]:
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.metrics.regression import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression, LassoCV, Lasso
from sklearn.ensemble import RandomForestRegressor

**Будем работать с набором данных по качеству белого вина (репозиторий UCI).**
**Загружаем данные.**

In [20]:
data = pd.read_csv('../../Labs/ml_lab06_regression_wine/winequality-white.csv', sep=';')

In [3]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
fixed acidity           4898 non-null float64
volatile acidity        4898 non-null float64
citric acid             4898 non-null float64
residual sugar          4898 non-null float64
chlorides               4898 non-null float64
free sulfur dioxide     4898 non-null float64
total sulfur dioxide    4898 non-null float64
density                 4898 non-null float64
pH                      4898 non-null float64
sulphates               4898 non-null float64
alcohol                 4898 non-null float64
quality                 4898 non-null int64
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


**Отделите целевой признак, разделите обучающую выборку в отношении 7:3 (30% - под оставленную выборку, пусть random_state=17) и отмасштабируйте данные с помощью StandardScaler.**

In [21]:
y = data['quality']
X = data.drop(['quality'], axis = 1)

X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=0.3, random_state=17)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train, y_train)
X_holdout_scaled = scaler.transform(X_holdout, y_holdout)

## Линейная регрессия

**Обучите простую линейную регрессию.**

In [22]:
linreg = LinearRegression()
linreg.fit(X_train, y_train)
linreg_train_preds = linreg.predict(X_train)
linreg_holdout_preds = linreg.predict(X_holdout)

**<font color='red'>Вопрос 1:</font> Каковы среднеквадратичные ошибки линейной регрессии на обучающей и отложенной выборках?**

In [23]:
print("Mean squared error (train): %.3f" % mean_squared_error(y_train, linreg_train_preds))
print("Mean squared error (test): %.3f"  % mean_squared_error(y_holdout, linreg_holdout_preds))

Mean squared error (train): 0.558
Mean squared error (test): 0.584


**Посмотрите на коэффициенты модели и отранжируйте признаки по влиянию на качество вина (учтите, что большие по модулю отрицательные значения коэффициентов тоже говорят о сильном влиянии). Создайте для этого новый небольшой DataFrame.**<br>
**<font color='red'>Вопрос 2:</font> Какой признак линейная регрессия считает наиболее сильно влияющим на качество вина?**

In [29]:
linreg.coef_

array([ 1.13730778e-01, -1.88687417e+00, -1.48507297e-03,  1.08182950e-01,
        3.52051235e-01,  2.43875224e-03,  3.33092994e-04, -2.27551202e+02,
        9.95829583e-01,  5.52494581e-01,  1.05264404e-01])

In [134]:
linreg_coef = pd.DataFrame(np.abs(linreg.coef_), X.columns)
linreg_coef.sort_values(by=0, ascending = False)

,0
density,227.551202
volatile acidity,1.886874
pH,0.995830
sulphates,0.552495
chlorides,0.352051
fixed acidity,0.113731
residual sugar,0.108183
alcohol,0.105264
free sulfur dioxide,0.002439
citric acid,0.001485


## Lasso-регрессия

**Обучите Lasso-регрессию с небольшим коэффициентом $\alpha = 0.01$ (слабая регуляризация). Пусть опять random_state=17.**

In [50]:
lasso1 = Lasso(alpha = 0.01, random_state = 17)
lasso1.fit(X,y) 

Lasso(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=17,
   selection='cyclic', tol=0.0001, warm_start=False)

**Посмотрите на коэффициенты модели и отранжируйте признаки по влиянию на качество вина. Какой признак "отвалился" первым, то есть наименее важен для объяснения целевого признака в модели Lasso?**

In [133]:
lasso1_coef = pd.DataFrame(np.abs(lasso1.coef_), X.columns)
lasso1_coef.sort_values(by = 0, ascending = False)

,0
volatile acidity,0.932822
alcohol,0.347006
fixed acidity,0.044986
residual sugar,0.020729
free sulfur dioxide,0.006324
total sulfur dioxide,0.001397
citric acid,0.000000
chlorides,0.000000
density,0.000000
pH,0.000000


**Теперь определите лучшее значение $\alpha$ в процессе кросс-валидации 5-кратной кросс-валидации. Используйте LassoCV и random_state=17.**

In [67]:
alphas = np.logspace(-6, 2, 200)
lasso_cv = LassoCV(alphas = alphas, random_state = 17, cv = 5)
lasso_cv.fit(X, y)

LassoCV(alphas=array([1.00000e-06, 1.09699e-06, ..., 9.11589e+01, 1.00000e+02]),
    copy_X=True, cv=5, eps=0.001, fit_intercept=True, max_iter=1000,
    n_alphas=100, n_jobs=None, normalize=False, positive=False,
    precompute='auto', random_state=17, selection='cyclic', tol=0.0001,
    verbose=False)

In [68]:
lasso_cv.alpha_

9.221978823334321e-06

**Выведите коэффициенты "лучшего" Lasso в порядке убывания влияния на качество вина.**<br>
**<font color='red'>Вопрос 3:</font> Какой признак "обнулился первым" в настроенной модели LASSO?**

In [135]:
lasso_cv_coef = pd.DataFrame(np.abs(lasso_cv.coef_), X.columns)
lasso_cv_coef.sort_values(by=0, ascending = False)

,0
density,121.175822
volatile acidity,1.881369
sulphates,0.589784
pH,0.585201
chlorides,0.361185
alcohol,0.226394
residual sugar,0.070675
fixed acidity,0.043108
citric acid,0.011173
free sulfur dioxide,0.003934


**Оцените среднеквадратичную ошибку модели на обучающей и тестовой выборках.**<br>
**<font color='red'>Вопрос 4:</font> Каковы среднеквадратичные ошибки настроенной LASSO-регрессии на обучающей и отложенной выборках?**

In [73]:
lasso_train_preds = lasso_cv.predict(X_train)
lasso_holdout_preds = lasso_cv.predict(X_holdout)

print("Mean squared error (train): %.3f" %  mean_squared_error(y_train, lasso_train_preds))
print("Mean squared error (test): %.3f"  %  mean_squared_error(y_holdout, lasso_holdout_preds))

Mean squared error (train): 0.561
Mean squared error (test): 0.568


## Случайный лес

**Обучите случайный лес с параметрами "из коробки", фиксируя только random_state=17.**

In [74]:
forest = RandomForestRegressor(random_state = 17)
forest.fit(X,y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=17, verbose=0, warm_start=False)

**<font color='red'>Вопрос 5:</font> Каковы среднеквадратичные ошибки случайного леса на обучающей выборке, на кросс-валидации (cross_val_score с scoring='neg_mean_squared_error' и остальными параметрами по умолчанию) и на отложенной выборке?**

In [124]:
forest_train_preds = forest.predict(X_train)
forest_holdout_preds = forest.predict(X_holdout)

print("Mean squared error (train): %.3f" % mean_squared_error(y_train, forest_train_preds))
print("Mean squared error (test): %.3f"  % mean_squared_error(y_holdout, forest_holdout_preds))
print(cross_val_score(forest, X, y, scoring = 'neg_mean_squared_error'))

Mean squared error (train): 0.068
Mean squared error (test): 0.070
[-0.70300061 -0.56942437 -0.54246324]


**Настройте параметры min_samples_leaf и max_depth с помощью GridSearchCV и опять проверьте качество модели на кросс-валидации и на отложенной выборке.**

In [128]:
forest_params = {'max_depth': list(range(10, 25)), 
                 'min_samples_leaf': list(range(1, 8)),
                 'max_features': list(range(6,12))}

locally_best_forest = GridSearchCV(forest, forest_params, n_jobs=-1)
locally_best_forest.fit(X, y)

GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=17, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'max_depth': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24], 'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7], 'max_features': [6, 7, 8, 9, 10, 11]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [129]:
locally_best_forest.best_params_, locally_best_forest.best_score_

({'max_depth': 10, 'max_features': 7, 'min_samples_leaf': 5},
 0.2857315909931129)

**К сожалению, результаты  GridSearchCV не полностью воспроизводимы (могут отличаться на разных платформах даже при фиксировании *random_state*). Поэтому обучите лес с параметрами max_depth=19, max_features=7, и min_samples_leaf=1 (лучшие в моем случае).**<br>
**<font color='red'>Вопрос 6:</font> Каковы среднеквадратичные ошибки настроенного случайного леса на обучающей выборке, на кросс-валидации (cross_val_score с scoring='neg_mean_squared_error') и на отложенной выборке?**

In [139]:
forest_with_params = RandomForestRegressor(max_depth=19, max_features=7, min_samples_leaf=1, random_state = 17)
forest_with_params.fit(X,y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=19,
           max_features=7, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=None, oob_score=False, random_state=17,
           verbose=0, warm_start=False)

In [143]:
forest_with_params_train_preds = forest_with_params.predict(X_train)
forest_with_params_holdout_preds = forest_with_params.predict(X_holdout)

print("Mean squared error (train): %.3f" % mean_squared_error(y_train, forest_with_params_train_preds))
print("Mean squared error (test): %.3f"  % mean_squared_error(y_holdout, forest_with_params_holdout_preds))
print(cross_val_score(forest_with_params, X, y, scoring = 'neg_mean_squared_error'))

Mean squared error (train): 0.077
Mean squared error (test): 0.073
[-0.71082219 -0.5784894  -0.53372768]


**Оцените важность признаков с помощью случайного леса.**<br>
**<font color='red'>Вопрос 7:</font> Какой признак оказался главным в настроенной модели случайного леса?**

In [145]:
rf_importance = pd.DataFrame(forest_with_params.feature_importances_, X.columns)
rf_importance.sort_values(by=0, ascending = False)

,0
alcohol,0.219346
volatile acidity,0.120421
free sulfur dioxide,0.112237
density,0.082157
residual sugar,0.075083
pH,0.071989
chlorides,0.070474
total sulfur dioxide,0.068492
fixed acidity,0.065793
citric acid,0.059751


**Сделайте выводы о качестве моделей и оценках влияния признаков на качество вина с помощью этих трех моделей.**

In [ ]:
"""Линейная и лассо регресии одинаково плохо себя показали в данной задаче
случайный лес отлично подошел для данной задачи, отталкиваясь от важности признаков в случайном лесу
можно сделать вывод что алкоголь имеет найбольшее влияние при оценке качества вина'''